# Clean HPS model with cross-validated subtyping
- take the full sample
- subsample using 10-fold cross-validation
- regress nuisance separately in the train and test fold
- extract subtypes on the train data
- extract weights for these subtypes from the train and test data
- predict easy cases on weights in train data
- predict ASD in test data using train data easy case signature
- report model performance

Other ideas:
- scale train data before nuisance regression
- then apply the fitted scaler to the test data before their nuisance regression?

In [25]:
%matplotlib inline

In [26]:
import os
import re
import sys
import time
import pickle
sys.path.append('/home/surchs/git/HPS')
from hps.predic import high_confidence
from hps.visu import hps_visu
sys.path.append('/home/surchs/git/HPS/examples/')
import warnings
warnings.filterwarnings('ignore')
import visu_demo
import scipy as sp
import patsy as pat
import numpy as np
import pandas as pd
import nibabel as nib
import sklearn as skl
import scipy.io as sio
import seaborn as sbn
import sklearn.metrics as skm
from scipy import cluster as scl
from scipy import stats as spt
from nilearn import plotting as nlp
from matplotlib import pyplot as plt
from sklearn import linear_model as sln
from sklearn import preprocessing as skp
from sklearn.model_selection import StratifiedKFold

In [27]:
n_seed = 12
n_subtypes = 4

In [28]:
# Paths
root_p = '/home/surchs/sim_big'
# Pheno
sample_p = os.path.join(root_p, 'PROJECT/abide_hps/pheno', 'psm_abide1.csv')
# Data
ct_p = os.path.join(root_p, 'PROJECT/abide_hps/ct')
seed_p = os.path.join(root_p, 'PROJECT/abide_hps/seed', 'MIST_{}'.format(n_seed))
mask_p = os.path.join(root_p, 'PROJECT/abide_hps/mask', 'MIST_mask.nii.gz')
label_p = os.path.join(root_p, 'ATLAS/MIST/Parcel_Information', 'MIST_{}.csv'.format(n_seed))
atlas_p = os.path.join(root_p, 'ATLAS/MIST/Parcellations/', 'MIST_{}.nii.gz'.format(n_seed))
# File templates
ct_t = '{}+{:07}_{}+{}_native_rms_rsl_tlaplace_30mm_{}.txt'
sd_t = 'sub_{{}}_mist_{}.npy'.format(n_seed)
# Make a temp save of the model
dump_p = os.path.join(root_p, 'PROJECT/abide_hps/single_dump_mist_{}.p'.format(n_seed))#_single_intercept
avg_dump_p = os.path.join(root_p, 'PROJECT/abide_hps/avg_dump_mist_{}_prereg_avg.p'.format(n_seed))#
out_p = os.path.join(root_p, 'PROJECT/abide_hps/ohbm_out')
train_resid_p = os.path.join(root_p, 'PROJECT/abide_hps/train_resid_{{}}_{}.npy'.format(n_seed))
train_sbt_p = os.path.join(root_p, 'PROJECT/abide_hps/train_sbt_{{}}_{}.npz'.format(n_seed))
test_resid_p = os.path.join(root_p, 'PROJECT/abide_hps/test_resid_{{}}_{}.npy'.format(n_seed))
if not os.path.isdir(out_p):
    os.makedirs(out_p)

In [29]:
# Load data
sample = pd.read_csv(sample_p)
sample['DX_CODE'] = sample['DX_GROUP'].replace({'Autism':1, 'Control':0})
label = pd.read_csv(label_p, delimiter=';')

In [30]:
mask_i = nib.load(mask_p)
mask = mask_i.get_data().astype(bool)
n_vox = np.sum(mask)
atlas = nib.load(atlas_p).get_data()

# Run the CV model

In [31]:
def corr2_coeff(A,B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:,None]
    B_mB = B - B.mean(1)[:,None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1);
    ssB = (B_mB**2).sum(1);

    # Finally get corr coeff
    return np.dot(A_mA,B_mB.T)/np.sqrt(np.dot(ssA[:,None],ssB[None]))

In [32]:
def subtype(stack, n_subtypes):
    # Normalize and then get the distance
    norm = skp.scale(stack, axis=1)
    # Get the lower triangle of the distance metric
    dist = sp.spatial.distance.pdist(norm)
    # Build the cluster
    link = scl.hierarchy.linkage(dist, method='ward')
    order = scl.hierarchy.dendrogram(link, no_plot=True)['leaves']
    part = scl.hierarchy.fcluster(link, n_subtypes, criterion='maxclust')
    return order, part, dist

In [33]:
def regress_fc(sample, formula, n_vox, n_seed, seed_p, sd_t):
    n_sub = sample.shape[0]
    resid_seed = np.zeros((n_sub, n_vox, n_seed))
    dmat_seed = pat.dmatrix(formula, data=sample)
    for sid in range(n_seed):
        # Build the regression model for the seed maps
        mod = sln.LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)
        sub_seed = np.zeros((n_sub, n_vox))
        # Line index doesn't necessarily match continuous index
        for rid, (rid_abs, row) in enumerate(sample.iterrows()):
            p = os.path.join(seed_p, sd_t.format(row['SUB_ID']))
            d = np.load(p)
            sub_seed[rid, :] = d[sid, ...]
        res = mod.fit(dmat_seed, sub_seed)
        resid = sub_seed - res.predict(dmat_seed)
        resid_seed[..., sid] = resid
    
    return resid_seed

In [34]:
def regress_ct(sample, formula, ct_p, ct_t):
    n_sub = sample.shape[0]
    # Generate the CT residuals
    for rid, (rid_abs, row) in enumerate(sample.iterrows()):
        p_right = os.path.join(ct_p, ct_t.format(row['Site'], row['Subject'], row['Session'], row['Run'], 'right'))
        p_left = os.path.join(ct_p, ct_t.format(row['Site'], row['Subject'], row['Session'], row['Run'], 'left'))
        ct_l = pd.read_csv(p_left, header=None)[0].values
        ct_r = pd.read_csv(p_right, header=None)[0].values
        # Combine left and right
        ct = np.concatenate((ct_l, ct_r))
        if rid==0:
            n_vert = len(ct)
            sub_ct = np.zeros((n_sub, n_vert))
        sub_ct[rid, :] = ct
    dmat_ct = pat.dmatrix(formula, data=sample)
    mod = sln.LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)
    res = mod.fit(dmat_ct, sub_ct)
    resid_ct = sub_ct - res.predict(dmat_ct)
    
    return resid_ct, res

In [35]:
def make_subtype_fc(resid, n_subtypes=5):
    n_sub, n_vox, n_seed = resid.shape
    # Run the FC subtypes
    weights_fc = np.zeros((n_sub, n_subtypes, n_seed))
    subtypes_fc = np.zeros((n_subtypes,) + resid.shape[1:])
    parts_fc = np.zeros((n_sub, n_seed))
    orders_fc = np.zeros((n_sub, n_seed))
    dists_fc = np.zeros((n_sub, n_sub, n_seed))

    for sid in range(n_seed):
        order_fc, part_fc, dist_fc = subtype(resid[..., sid], n_subtypes)
        dists_fc[..., sid] = sp.spatial.distance.squareform(dist_fc)
        parts_fc[:, sid] = part_fc
        orders_fc[:, sid] = order_fc
        # Make the subtypes
        subtypes_fc_tmp = np.array([np.mean(resid[part_fc==i, :, sid], 0) 
                                    for i in range(1,n_subtypes+1)])
        subtypes_fc[..., sid] = subtypes_fc_tmp
        # Compute the weights
        weights_fc[..., sid] = corr2_coeff(resid[..., sid], subtypes_fc_tmp)
    return subtypes_fc, weights_fc

In [36]:
def make_subtype_ct(resid, n_subtypes):
    order_ct, part_ct, dist_ct = subtype(resid, n_subtypes)
    # Make the subtypes
    subtypes_ct = np.array([np.mean(resid[part_ct==i, :], 0) 
                            for i in range(1,n_subtypes+1)])
    # Compute the weights
    weights_ct = corr2_coeff(resid, subtypes_ct)
    return (subtypes_ct, weights_ct)

In [37]:
def make_weights_fc(subtypes, resid):
    n_sub, n_vox, n_seed = resid.shape
    n_subtypes = subtypes.shape[0]
    weights_fc = np.zeros((n_sub, n_subtypes, n_seed))
    for sid in range(n_seed):
    # Compute the weights
        weights_fc[..., sid] = corr2_coeff(resid[..., sid], subtypes[..., sid])
    return weights_fc

In [38]:
def make_weights_ct(subtypes, resid):
    weights_ct = corr2_coeff(resid, subtypes)
    return weights_ct

In [39]:
# Get the full range of subject indices and clinical labels
sub_indices = sample.index.values
labels = sample['DX_CODE'].values

In [40]:
fc_cols = ['fc_n{}_s{}'.format(nid+1, sid+1) 
           for sid in range(n_subtypes) 
           for nid in range(n_seed)]
fc_col_params = [(nid, sid) for sid in range(n_subtypes) 
                 for nid in range(n_seed)]

ct_cols = ['ct_s{}'.format(sid+1) 
           for sid in range(n_subtypes)]
ct_col_params = [(-1, sid) for sid in range(n_subtypes)]
cols = ct_cols + fc_cols
#col_features = ['BV', 'AGE_AT_SCAN', 'FD_scrubbed', ] + cols
#col_features = ['BV', 'AGE_AT_SCAN'] + cols
col_features = cols
#col_params = [(None, None)]*3 + ct_col_params + fc_col_params
#col_params = [(None, None)]*2 + ct_col_params + fc_col_params
col_params = ct_col_params + fc_col_params

In [41]:
scores_s1_l = list()
scores_s2_l = list()
y_target_l = list()

start = time.time()
took = []
skf = StratifiedKFold(n_splits=10)
for cv_idx, (train_index, test_index) in enumerate(skf.split(sub_indices, labels)):
    cv_start = time.time()
    
    train_p = train_resid_p.format(cv_idx)
    sbt_p = train_sbt_p.format(cv_idx)
    test_p = test_resid_p.format(cv_idx)
    
    # Get the train, and test sample
    train_sample = sample.loc[train_index]
    test_sample = sample.loc[test_index]
    n_sub_train = train_sample.shape[0]
    n_sub_test = test_sample.shape[0]
    
    # Replicate the subtyping process
    # Extract the train and test data and regress nuisance factors
    if not os.path.isfile(train_p):
        train_resid_fc = regress_fc(train_sample, 
                                   'AGE_AT_SCAN + FD_scrubbed + Site', 
                                   n_vox, n_seed=n_seed, 
                                   seed_p=seed_p, sd_t=sd_t)

        np.save(train_p, train_resid_fc)
    else:
        train_resid_fc = np.load(train_p)
        
    if not os.path.isfile(test_p):
        test_resid_fc = regress_fc(test_sample, 
                                  'AGE_AT_SCAN + FD_scrubbed + Site', 
                                  n_vox, n_seed=n_seed, 
                                  seed_p=seed_p, sd_t=sd_t)
        np.save(test_p, test_resid_fc)
    else:
        test_resid_fc = np.load(test_p)
        
    (train_resid_ct, mod_ct_train) = regress_ct(train_sample, 'AGE_AT_SCAN + Site', ct_p, ct_t)
    (test_resid_ct, mod_ct_test) = regress_ct(test_sample, 'AGE_AT_SCAN + Site', ct_p, ct_t)
    # Make the subtypes from the train data
    
    
    if not os.path.isfile(sbt_p):
        (subtypes_fc, train_weights_fc) = make_subtype_fc(train_resid_fc, n_subtypes=n_subtypes)
        np.savez(sbt_p, subtypes_fc=subtypes_fc, train_weights_fc=train_weights_fc)
    else:
        print('loading sbt from {}'.format(sbt_p))
        tmp = np.load(sbt_p)
        subtypes_fc = tmp['subtypes_fc']
        train_weights_fc = tmp['train_weights_fc']
    
    (subtypes_ct, train_weights_ct) = make_subtype_ct(train_resid_ct, n_subtypes=n_subtypes)
    # Get the test weights
    test_weights_fc = make_weights_fc(subtypes_fc, test_resid_fc)
    test_weights_ct = make_weights_ct(subtypes_ct, test_resid_ct)
    
    # Build input data
    train_fc = np.reshape(train_weights_fc, (n_sub_train, n_subtypes*n_seed))
    test_fc = np.reshape(test_weights_fc, (n_sub_test, n_subtypes*n_seed))
    train_w = np.concatenate((train_weights_ct, train_fc), 1)
    test_w = np.concatenate((test_weights_ct, test_fc), 1)
    
    # Make sure we use the correct index or else there will be NaNs in the weight columns
    w_data_train = pd.DataFrame(data=train_w, columns=cols, index=train_index)
    data_train = train_sample.join(w_data_train)
    w_data_test = pd.DataFrame(data=test_w, columns=cols, index=test_index)
    data_test = test_sample.join(w_data_test)
    
    # Select the features
    scaler = skl.preprocessing.StandardScaler()
    x_train = data_train.loc[:, col_features]
    # Normalize
    X_train = scaler.fit_transform(x_train)
    # Take the numeric diagnosis code, 0 is control, 1 is autism
    y_train = data_train.loc[:, ['DX_CODE']].values.squeeze()

    # Same for the test data
    x_test = data_test.loc[:, col_features]
    # Normalize, but use the fitted scalar of the training data
    X_test = scaler.transform(x_test)
    y_test = data_test.loc[:, ['DX_CODE']].values.squeeze()
    
    # Train the model
    hps = high_confidence.TwoStagesPrediction(verbose=False,
                                          n_iter=200,
                                          shuffle_test_split=0.5,
                                            gamma=1,
                                          min_gamma=0.95,
                                          thresh_ratio=0.8)
    hps.fit(X_train, y_train)
    scores, dic_results = hps.predict(X_test)
    scores_s1_l.append(dic_results['s1_hat'])
    scores_s2_l.append(dic_results['s2_hat'])
    y_target_l.append(y_test)

    current_duration = time.time()-cv_start
    took.append(current_duration)
    avg_time = np.mean(took)
    elapsed_time = np.sum(took)
    remaining_time = avg_time * (9-cv_idx)
    
    print('CV fold {} done. Took {:.2f}s ({:.2f}s), {:.2f}s total, {:.2f}s to go.'.format(cv_idx+1,
                                                                              current_duration,
                                                                              avg_time,
                                                                              elapsed_time,
                                                                              remaining_time))

loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_0_12.npz
Stage 1
Stage 2
CV fold 1 done. Took 144.99s (144.99s), 144.99s total, 1304.92s to go.
loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_1_12.npz
Stage 1
Stage 2
CV fold 2 done. Took 142.30s (143.64s), 287.29s total, 1149.15s to go.
loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_2_12.npz
Stage 1
Stage 2
CV fold 3 done. Took 164.11s (150.47s), 451.40s total, 1053.26s to go.
loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_3_12.npz
Stage 1
Stage 2
CV fold 4 done. Took 165.06s (154.11s), 616.46s total, 924.68s to go.
loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_4_12.npz
Stage 1
Stage 2
CV fold 5 done. Took 151.66s (153.62s), 768.12s total, 768.12s to go.
loading sbt from /home/surchs/sim_big/PROJECT/abide_hps/train_sbt_5_12.npz
Stage 1
Stage 2
CV fold 6 done. Took 165.92s (155.67s), 934.03s total, 622.69s to go.
loading sbt from /home/surchs/s

In [42]:
y = sample.DX_CODE.values.squeeze()
ohe = skl.preprocessing.OneHotEncoder(sparse=False)
ohe.fit(y.reshape(-1, 1))
labels = ohe.transform(y.reshape(-1, 1))

scores_s1_arr = np.vstack(scores_s1_l)
scores_s2_arr = np.vstack(scores_s2_l)
y_target_arr = np.hstack(y_target_l)

########################
print('##########################')
# S1
y_mb = ohe.transform(y_target_arr[:,np.newaxis])
pred_y_ = scores_s1_arr

print('Stage 1 (BASE)')
hps_visu.print_scores(hps_visu.scores(y_mb, pred_y_))


# S2
y_mb = ohe.transform(y_target_arr[:,np.newaxis])
pred_y_ = scores_s2_arr

print('Stage 2 (HPS)')
hps_visu.print_scores(hps_visu.scores(y_mb, pred_y_)) 
print('##########################')

##########################
Stage 1 (BASE)
Class 0 Precision: 58.12 Specificity: 56.04 Recall: 59.04 N: 191
Class 1 Precision: 56.98 Specificity: 59.04 Recall: 56.04 N: 179
Total Precision: 57.55 Specificity: 57.54 Recall: 57.54 N: 185
Stage 2 (HPS)
Class 0 Precision: 67.65 Specificity: 93.96 Recall: 12.23 N: 34
Class 1 Precision: 83.87 Specificity: 97.34 Recall: 14.29 N: 31
Total Precision: 75.76 Specificity: 95.65 Recall: 13.26 N: 32
##########################


- all sites, only brain: Precision: 80.00 Specificity: 96.81 Recall: 13.19 N: 30
- 'BV', 'AGE_AT_SCAN': Precision: 78.38 Specificity: 95.74 Recall: 15.93 N: 37
- ['BV', 'AGE_AT_SCAN']: Precision: 90.91 Specificity: 99.47 Recall:  5.49 N: 11, gamma=1
- min_gamma=0.95, thresh_ratio=0.4: Precision: 78.38 Specificity: 95.74 Recall: 15.93 N: 37
- gamma=1,min_gamma=0.98,thresh_ratio=0.4: Precision: 100.00 Specificity: 100.00 Recall:  8.79 N: 16
- only brain: Precision: 83.33 Specificity: 97.87 Recall: 10.99 N: 24
- iter = 1000: Precision: 78.26 Specificity: 97.34 Recall:  9.89 N: 23
- thresh: 0.6: Precision: 79.17 Specificity: 97.34 Recall: 10.44 N: 24

In [22]:
a = np.load('/home/surchs/sim_big/PROJECT/abide_hps/train_sbt_0_12.npz')
aa = a['train_weights_fc']
b = np.load('/home/surchs/sim_big/PROJECT/abide_hps/train_sbt_0_12_onlybrain.npz')
bb = b['train_weights_fc']

In [23]:
aa.shape

(332, 4, 12)

In [24]:
bb.shape

(332, 4, 12)

In [20]:
a

## Train a model on the whole dataset

In [ ]:
n_sub = sample.shape[0]


print('Need to Run all')
# Replicate the subtyping process
# Extract the train and test data and regress nuisance factors
resid_fc = regress_fc(sample, 
                           'AGE_AT_SCAN + FD_scrubbed + Site', 
                           n_vox, n_seed=n_seed, 
                           seed_p=seed_p, sd_t=sd_t)
(resid_ct, mod_ct) = regress_ct(sample, 'AGE_AT_SCAN + Site', ct_p, ct_t)

# Make the subtypes from the train data
(subtypes_fc, weights_fc) = make_subtype_fc(resid_fc, n_subtypes=n_subtypes)
(subtypes_ct, weights_ct) = make_subtype_ct(resid_ct, n_subtypes=n_subtypes)

# Build input data
fc = np.reshape(weights_fc, (n_sub, n_subtypes*n_seed))

features = np.concatenate((weights_ct, fc), 1)


# Make sure we use the correct index or else there will be NaNs in the weight columns
feat_data = pd.DataFrame(data=features, columns=cols)
data = sample.join(feat_data)

# Select the features
scaler = skl.preprocessing.StandardScaler()
x_ = data.loc[:, col_features]
# Normalize
X = scaler.fit_transform(x_)
# Take the numeric diagnosis code, 0 is control, 1 is autism
y = data.loc[:, ['DX_CODE']].values.squeeze()

# Train the model
hps = high_confidence.TwoStagesPrediction(verbose=False,
                                      n_iter=1000,
                                      shuffle_test_split=0.5,
                                        gamma=1,
                                      min_gamma=0.95,
                                      thresh_ratio=0.3)


hps.fit(X, y)
res_hitproba = hps.training_hit_probability
plt.figure()
plt.title('Class 0 hit probability distribution')
plt.hist(hps.training_hit_probability[y==0],10);
plt.figure()
plt.title('Class 1 hit probability distribution')
plt.hist(hps.training_hit_probability[y==1],10);

In [ ]:
hps_base_rate = np.sum(hps.training_hit_probability[y==1]>0.9)/len(hps.training_hit_probability[y==1])
print('HPS base rate {:.2f}%'.format(hps_base_rate*100))

## Review the contributing FC features

In [ ]:
# Get the feature weights of the second stage for class 2 (ASD)
feature_weights = hps.confidencemodel.clfs[1].coef_
non_zero_features = np.where(feature_weights!=0)[1]

In [ ]:
f_dc = {key:list() for key in ['net', 'sid', 'weight', 'name', 'net_name', 'kind', 'nonzero', 'risk']}
for i in range(len(col_features)):
    f_name = col_features[i]
    if 'fc' in f_name:
        # FC
        net = col_params[i][0]
        sid = col_params[i][1]
        net_name = label.loc[label.roi==net+1].name.values[0]
        kind = 'FC'
    elif 'ct' in f_name:
        # CT
        net = None
        sid = col_params[i][1]
        net_name = None
        kind = 'CT'
    else:
        # Behaviour
        net = None
        sid = None
        net_name = None
        net_name = None
        kind = 'Other'
    weight = feature_weights.flatten()[i]
    
    
    f_dc['net'].append(net)
    f_dc['sid'].append(sid)
    f_dc['weight'].append(weight)
    f_dc['name'].append(f_name)
    f_dc['net_name'].append(net_name)
    f_dc['kind'].append(kind)
    f_dc['nonzero'].append(weight!=0)
    f_dc['risk'].append(weight>0)
feature_data = pd.DataFrame(f_dc)
feature_nonzero = feature_data.loc[feature_data.nonzero]

In [ ]:
f_sorted = feature_nonzero.sort_values(by=['net', 'sid'])
f_sorted

# No additional features

In [ ]:
np.sum(feature_data.nonzero)

In [ ]:
f_sorted = feature_nonzero.sort_values(by=['net', 'sid'])
tmp_feat_sort_ind = f_sorted.index.values
f_sorted.reset_index(drop=True, inplace=True)

net_ind = list()
net_border = list()
lbl = list()
rbl = list()
for nid in range(n_seed):
    # Find the networks hits
    h_ind = f_sorted.loc[f_sorted.net==nid].index.values
    if not h_ind[0]-0.5 in net_border:
        net_border.append(h_ind[0]-0.5)
    net_border.append(h_ind[-1]+0.5)
    lbl.append( h_ind[0]-0.5)
    rbl.append( h_ind[-1]+0.5)
        
    if len(h_ind)==2:
        net_ind.append(h_ind[0]+0.5)
    else:
        net_ind.append(h_ind[1])
# Add CT and Other
net_ind.append(35)
net_ind.append(38)
c_ind = np.array(net_ind)
b_ind = np.array(net_border)
lb = np.array(lbl)
rb = np.array(rbl)

In [ ]:
b_ind

In [ ]:
c_ind

In [ ]:
f_sorted.net.unique()

In [ ]:
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost

In [ ]:
names = list(f_sorted.iloc[c_ind].net_name.values)
names[-2] = 'Cortical Thickness'
names[-1] = 'Other'

In [ ]:
# Re-order the networks so the effects don't overlap
new_net_order = ['AUDITORY_NETWORK_and_POSTERIOR_INSULA',
                 'MESOLIMBIC_NETWORK',
                 'DEFAULT_MODE_NETWORK_lateral',
                 'SOMATOMOTOR_NETWORK',
                 'DEFAULT_MODE_NETWORK_posteromedial',
                 'DEFAULT_MODE_NETWORK_anteromedial_and_left_ANGULAR_GYRUS',
                 'BASAL_GANGLIA_and_THALAMUS',
                 'VENTRAL_VISUAL_STREAM_and_DORSAL_VISUAL_STREAM',
                 'FRONTO_PARIETAL_NETWORK',
                 'VENTRAL_ATTENTION_NETWORK_and_SALIENCE_NETWORK',
                 'VISUAL_NETWORK',
                 'CEREBELLUM']
new_net_names = new_net_order + ['Cortical Thickness', 'Other']

In [ ]:
net_ind_l = list()
# Get the FC indices
for i in new_net_order:
    for j in f_sorted.loc[f_sorted.net_name==i].index.values:
        net_ind_l.append(j)
# Add the other indices
for j in f_sorted.loc[f_sorted.net_name.isnull()].index.values:
    net_ind_l.append(j)
new_net_ind = np.array(net_ind_l)

In [ ]:
# Show all the non-zero data

f = plt.figure(figsize=(3,15), constrained_layout=True)
#ax = SubplotHost(f, 111)
ax = f.add_subplot(111)
for idx, (l, r) in enumerate(zip(lbl, rbl)):
    if idx%2==0:
        ax.axhspan(l, r, facecolor='lightgrey', alpha=0.2)
    else:
        ax.axhspan(l, r, facecolor='grey', alpha=0.2)
    ax.axhspan(33.5, 36.5, facecolor='lightblue', alpha=0.2)
    ax.axhspan(36.5, 39.5, facecolor='darkgrey', alpha=0.2)
#f.add_subplot(ax)
#offset = 0, -25
#ax2 = ax.twiny()
#g = sbn.barplot(x='feature', y='weights', data=weights, hue='feature_type', ax=ax)
g = sbn.barplot(x='weight', y='name', data=f_sorted.iloc[new_net_ind], hue='kind', ax=ax, ci=None, dodge=False, 
                palette=sbn.xkcd_palette(['yellow orange', 'cerulean', 'light grey']), orient='h')
#g.set(xlabel='Feature Weights of HPS model', ylabel='')
sbn.despine(left=True);
#ax.set_xticklabels(ax.get_xticklabels(), rotation=90);
ax.set_yticks(c_ind)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticklabels([])
ax.set_yticks([])
ax.axvline(0, color='black')
#for i in b_ind:
#    ax.axvline(i, ymin=0.4, ymax=0.5, color='grey')
#new_axisline = ax2.get_grid_helper().new_fixed_axis
#ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax.legend_.remove()

f.savefig(os.path.join(out_p, 'non-zero-features_no_name.png'), dpi=300, transparent=True)

In [ ]:
asd_indices = sample.loc[y==1].index.values
hps_indices = asd_indices[np.where(hps.training_hit_probability[y==1]>0.95)]
nonhps_indices = asd_indices[np.where(hps.training_hit_probability[y==1]<=0.95)]
cond_arr = np.zeros(n_sub)
cond_arr[hps_indices] = 2
cond_arr[nonhps_indices] = 1

In [ ]:
# Make the traces
features_all = data.loc[:, col_features].values
# Sort features to what I use in the rest of the plots
features_resort_like_others = features_all[:, tmp_feat_sort_ind]

feat_scaler = skl.preprocessing.StandardScaler()
features_all_scaled = feat_scaler.fit_transform(features_resort_like_others)
n_nonzero_features = features_resort_like_others.shape[1]
nonzero_feat_name = f_sorted.name.values
# Resort the features again according to the new networks

features_all_scaled_resorted = features_all_scaled[:, new_net_ind]
feat_names_resorted = list(nonzero_feat_name[new_net_ind])

trace_data = {'sub':[rid for rid, row in sample.iterrows() for fid in range(n_nonzero_features)],
              'cond':[cond_arr[rid] for rid, row in sample.iterrows() for fid in range(n_nonzero_features)],
              'weight':[features_all_scaled_resorted[rid, fid] for rid, row in sample.iterrows() for fid in range(n_nonzero_features)],
              'feat':[fid for rid, row in sample.iterrows() for fid in range(n_nonzero_features)]}
trace = pd.DataFrame(trace_data)
trace['condition'] = trace['cond'].replace({0:'TDC', 1:'iASD', 2:'pASD'})

In [ ]:
ff_names = [col_features[i] for i in nonzero_feat_ind[new_net_ind]]

f = plt.figure(figsize=(20,10), constrained_layout=True)
#ax = SubplotHost(f, 111)
ax = f.add_subplot(111)
qq = sbn.tsplot(time="feat", value="weight",
           unit="sub", condition="condition",data=trace, ci=[10,50,90], ax=ax)
ax.set_xticks(np.arange(n_nonzero_features))
ax.set_xticklabels(feat_names_resorted, rotation=90);

In [ ]:
# Show all the non-zero data

f = plt.figure(figsize=(10,20), constrained_layout=True)
#ax = SubplotHost(f, 111)
ax = f.add_subplot(111)
for idx, (l, r) in enumerate(zip(lbl, rbl)):
    if idx%2==0:
        ax.axhspan(l, r, facecolor='lightgrey', alpha=0.2)
    else:
        ax.axhspan(l, r, facecolor='grey', alpha=0.2)
    ax.axhspan(33.5, 36.5, facecolor='lightblue', alpha=0.2)
    ax.axhspan(36.5, 39.5, facecolor='darkgrey', alpha=0.2)
#f.add_subplot(ax)
#offset = 0, -25
#ax2 = ax.twiny()
#g = sbn.barplot(x='feature', y='weights', data=weights, hue='feature_type', ax=ax)
g = sbn.barplot(x='weight', y='name', data=f_sorted.iloc[new_net_ind], hue='kind', ax=ax, ci=None, dodge=False, 
                palette=sbn.xkcd_palette(['yellow orange', 'cerulean', 'light grey']), orient='h')
g.set(xlabel='Feature Weights of HPS model', ylabel='')
sbn.despine();
#ax.set_xticklabels(ax.get_xticklabels(), rotation=90);
ax.set_yticks(c_ind)
ax.set_yticklabels(new_net_names, rotation=0)
ax.axvline(0, color='black')
#for i in b_ind:
#    ax.axvline(i, ymin=0.4, ymax=0.5, color='grey')
#new_axisline = ax2.get_grid_helper().new_fixed_axis
#ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax.legend_.remove()

f.savefig(os.path.join(out_p, 'non-zero-features.png'), dpi=300, transparent=True)

In [ ]:
lbl

In [ ]:
rbl

In [ ]:
# Show all the non-zero data
f = plt.figure(figsize=(15.5,4), constrained_layout=True)
#ax = SubplotHost(f, 111)
ax = f.add_subplot(111)
ax2 = ax.twinx()


ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['bottom'].set_visible(False)

for idx, (l, r) in enumerate(zip(lbl, rbl)):
    if idx%2==0:
        ax.axvspan(l, r, facecolor='lightgrey', alpha=0.2)
    else:
        ax.axvspan(l, r, facecolor='grey', alpha=0.2)
ax.axvspan(33.5, 36.5, facecolor='lightgrey', alpha=0.2)
ax.axvspan(36.5, 39.5, facecolor='grey', alpha=0.2)

#g = sbn.barplot(x='feature', y='weights', data=weights, hue='feature_type', ax=ax)
g = sbn.barplot(x='name', y='weight', data=f_sorted.iloc[new_net_ind], hue='kind', ax=ax, ci=None, dodge=False, 
                edgecolor="black", facecolor='white')
                #palette=sbn.xkcd_palette(['yellow orange', 'cerulean', 'light grey']), orient='v') #
#g.set(xlabel='Feature Weights of HPS model', ylabel='')

qq = sbn.tsplot(time="feat", value="weight",
           unit="sub", condition="condition",data=trace, ci=[95], ax=ax2,
               color=sbn.xkcd_palette(['green', 'cerulean', 'gold']), alpha=0.5)
#ax.set_xticks(np.arange(n_nonzero_features))
#ax.set_xticklabels(feat_names_resorted, rotation=90);
#sbn.despine(bottom=True);

#ax.set_yticks(c_ind)
#ax.set_yticklabels(names, rotation=0)
ax.axhline(0, color='black')
#for i in b_ind:
#    ax.axvline(i, ymin=0.4, ymax=0.5, color='grey')
#new_axisline = ax2.get_grid_helper().new_fixed_axis
#ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax.legend_.remove()
ax.set_xticks(np.arange(n_nonzero_features)[-3:]);
ax.set_xticklabels(['Brain volume', 'Age', 'Head motion'], rotation=90)
ax.set_xlabel('');
ax.set_ylabel('Feature weight');
ax2.set_ylabel('Feature loading');
ax2.legend(loc='center left', bbox_to_anchor=(0.93, 0.85))
ax.set_xlim([-0.5, 39.5])


f.savefig(os.path.join(out_p, 'non-zero-features_horizontal.png'), dpi=300, transparent=True)

In [ ]:
fc_data = feature_data.loc[feature_data.kind=='FC']
fc_data.sort_values(by='weight', inplace=True)
fc_data[['net', 'sid']] = fc_data[['net', 'sid']].astype(int)

In [ ]:
# How many networks are involved in the bottom 5 features
protective_fc = fc_data.iloc[:5]['net'].unique()

In [ ]:
fc_data.iloc[:5]

In [ ]:
# How many networks are involved in the top 5 features
risk_fc = fc_data.iloc[::-1][:5]['net'].unique()

In [ ]:
fc_data.iloc[::-1][:5]

In [ ]:
set(risk_fc).union(protective_fc)

In [ ]:
len(set(risk_fc).union(protective_fc))

In [ ]:
# Show me the top 5 features
for rid, (tmp, row) in enumerate(fc_data.iloc[::-1][:5].iterrows()):
    f = plt.figure(figsize=(8,4), constrained_layout=True)
    ax1 = f.add_subplot(111) 
    #ax2 = f.add_subplot(122)
    # Show me the subtype
    n_name = label.loc[label.roi==row['net']+1].name.values[0]
    vol = np.zeros(mask.shape)
    vol[mask] = subtypes_fc[row['sid'], :, row['net']]
    img = nib.Nifti1Image(vol, affine=mask_i.affine)
    
    a_vol = np.zeros(mask.shape)
    a_vol[atlas==row['net']+1] = 1
    a_img = nib.Nifti1Image(a_vol, affine=mask_i.affine)
    
    display = nlp.plot_stat_map(img, cut_coords=(0,0,0), axes=ax1, colorbar=False, draw_cross=False, cmap=plt.cm.RdBu_r)
    display.add_contours(a_img, levels=[.5], colors='black') 
    
    a_vol = np.zeros(mask.shape)
    a_vol[atlas==row['net']+1] = 1
    a_img = nib.Nifti1Image(a_vol, affine=mask_i.affine)
    f.savefig(os.path.join(out_p, 'fc_sbt_risk_{}_{}_{}.png'.format(rid, row['sid']+1, n_name)), dpi=300, transparent=True)

In [ ]:
# Show me the top 5 features
for rid, (tmp, row) in enumerate(fc_data.iloc[:5].iterrows()):
    f = plt.figure(figsize=(8,4), constrained_layout=True)
    ax1 = f.add_subplot(111) 
    #ax2 = f.add_subplot(122)
    # Show me the subtype
    n_name = label.loc[label.roi==row['net']+1].label.values[0]
    vol = np.zeros(mask.shape)
    vol[mask] = subtypes_fc[row['sid'], :, row['net']]
    img = nib.Nifti1Image(vol, affine=mask_i.affine)
    
    a_vol = np.zeros(mask.shape)
    a_vol[atlas==row['net']+1] = 1
    a_img = nib.Nifti1Image(a_vol, affine=mask_i.affine)
    
    display = nlp.plot_stat_map(img, cut_coords=(0,0,0), axes=ax1, colorbar=False, draw_cross=False, cmap=plt.cm.RdBu_r)
    display.add_contours(a_img, levels=[.5], colors='black') 
    
    a_vol = np.zeros(mask.shape)
    a_vol[atlas==row['net']+1] = 1
    a_img = nib.Nifti1Image(a_vol, affine=mask_i.affine)
    f.savefig(os.path.join(out_p, 'fc_sbt_protective_{}_{}_{}.png'.format(rid, row['sid']+1, n_name)), dpi=300, transparent=True)

## Save the average maps

In [ ]:
avg_fc.shape

In [ ]:
np.min(avg_fc, 1)

In [ ]:
for nid in range(n_seed):
    f = plt.figure(figsize=(8,4))
    ax1 = f.add_subplot(111) 
    #ax2 = f.add_subplot(122)
    # Show me the subtype
    n_name = label.loc[label.roi==nid+1].label.values[0]
    vol = np.zeros(mask.shape)
    vol[mask] = avg_fc[:, nid]
    img = nib.Nifti1Image(vol, affine=mask_i.affine)
    
    a_vol = np.zeros(mask.shape)
    a_vol[atlas==nid+1] = 1
    a_img = nib.Nifti1Image(a_vol, affine=mask_i.affine)
    
    display = nlp.plot_anat(cut_coords=(0,0,0), axes=ax1, draw_cross=False)
    display.add_overlay(img, cmap=plt.cm.viridis, colorbar=False, vmin=0)
    display.add_contours(a_img, levels=[.5], colors='black')
    f.savefig(os.path.join(out_p, 'fc_avg_{}.png'.format(n_name)), dpi=300, transparent=True)

In [ ]:
# Get the CT subtypes
for i in range(n_subtypes):
    ct_l = subtypes_ct[i, :40962]
    ct_r = subtypes_ct[i, :40962]
    np.savetxt(os.path.join(out_p, 'ct_{}_left.txt'.format(i)),ct_l)
    np.savetxt(os.path.join(out_p, 'ct_{}_right.txt'.format(i)),ct_r)

## Get best slice for network

In [ ]:
fc_data.loc[:4]

## Which are the features that are not zero

## Make the average positive feature (first 10)

In [ ]:
for i in sort_idx[:10]:
    print(col_params[i])

In [ ]:
pos_feat = np.zeros(subtypes_fc.shape[1])
q = 0
for i in sort_idx[-3:]:
    tmp = col_params[i]
    if feature_weights[:, i]<0:
        continue
    if tmp[0] is None:
        continue
    if tmp[0]<0:
        continue
    pos_feat += subtypes_fc[tmp[1], :, tmp[0]]
    q+=1
pos_feat = pos_feat/q


pos_vol = np.zeros(mask.shape)
pos_vol[mask] = pos_feat
pos_img = nib.Nifti1Image(pos_vol, affine=mask_i.affine)
nlp.plot_stat_map(pos_img, cut_coords=(0,0,0))

In [ ]:
feature_weights.shape

In [ ]:
neg_feat = np.zeros(subtypes_fc.shape[1])
q = 0
for i in sort_idx[:10]:
    tmp = col_params[i]
    if feature_weights[:, i]>0:
        continue
    if tmp[0] is None:
        continue
    if tmp[0]<0:
        continue
    neg_feat += subtypes_fc[tmp[1], :, tmp[0]]
    q+=1
neg_feat = neg_feat/q


neg_vol = np.zeros(mask.shape)
neg_vol[mask] = neg_feat
neg_img = nib.Nifti1Image(neg_vol, affine=mask_i.affine)
nlp.plot_stat_map(neg_img, cut_coords=(0,0,0))

## Attempt a validation

In [ ]:
y = sample.DX_CODE.values.squeeze()
ohe = skl.preprocessing.OneHotEncoder(sparse=False)
ohe.fit(y.reshape(-1, 1))
labels = ohe.transform(y.reshape(-1, 1))

In [ ]:
# Load data
validation_p = os.path.join(root_p, 'PROJECT/abide_hps/pheno', 'validation_abide1.csv')
validation = pd.read_csv(validation_p)
validation['DX_CODE'] = validation['DX_GROUP'].replace({'Autism':1, 'Control':0})
n_sub_valid = validation.shape[0]

In [ ]:
valid_resid_fc = regress_fc(validation, 
                            'AGE_AT_SCAN + FD_scrubbed + Site', 
                            n_vox, n_seed=n_seed, 
                            seed_p=seed_p, sd_t=sd_t)
(valid_resid_ct, valid_mod_ct) = regress_ct(validation, 'AGE_AT_SCAN + Site', ct_p, ct_t)
valid_weights_fc = make_weights_fc(subtypes_fc[0], valid_resid_fc)
valid_weights_ct = make_weights_ct(subtypes_ct[0], valid_resid_ct)

valid_fc = np.reshape(valid_weights_fc, (n_sub_valid, n_subtypes*n_seed))
valid_w = np.concatenate((valid_weights_ct, valid_fc), 1)

w_data_valid = pd.DataFrame(data=valid_w, columns=cols)
data_valid = validation.join(w_data_valid)

In [ ]:
x_valid = data_valid.loc[:, col_features]
X_valid = scaler.transform(x_valid)
y_valid = data_valid.loc[:, ['DX_CODE']].values.squeeze()

scores_valid, dic_results_valid = hps.predict(X_valid)

In [ ]:
scores_s1_valid = dic_results_valid['s1_hat']
scores_s2_valid = dic_results_valid['s2_hat']

In [ ]:
########################
print('##########################')
# S1
y_mb_valid = ohe.transform(y_valid[:,np.newaxis])
pred_y_valid = scores_s1_valid

print('Stage 1 (BASE)')
hps_visu.print_scores(hps_visu.scores(y_mb_valid, pred_y_valid))


# S2
y_mb = ohe.transform(y_valid[:,np.newaxis])
pred_y_valid = scores_s2_valid

print('Stage 2 (HPS)')
hps_visu.print_scores(hps_visu.scores(y_mb_valid, pred_y_valid)) 
print('##########################')

Well, that isn't so amazing now, is it? But it's also not worlds apart in terms of specificity. I mean the precision suffers greatly and overall the model doesn't appear that nice anymore. But it's not so horrible now.

In [ ]:
tn, fp, fn, tp = skl.metrics.confusion_matrix(y_mb_valid[:, 1], scores_s2_valid[: , 1]).ravel().astype(float)

In [ ]:
tp

In [ ]:
fp

In [ ]:
tn

In [ ]:
fn

## Tell me something about these guys

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
HPS_ind_valid = pred_y_valid[:, 1]==1

In [ ]:
# Give me three classes here
asd_hps_valid = [True if HPS_ind_valid[rid]==1 and row['DX_GROUP']=='Autism' else False for rid, row in validation.iterrows()]
asd_ns_valid = [True if HPS_ind_valid[rid]==0 and row['DX_GROUP']=='Autism' else False for rid, row in validation.iterrows()]
tdc_hps_valid = [True if HPS_ind_valid[rid]==1 and row['DX_GROUP']=='Control' else False for rid, row in validation.iterrows()]
tdc_ns_valid = [True if HPS_ind_valid[rid]==0 and row['DX_GROUP']=='Control' else False for rid, row in validation.iterrows()]
group = list()
for rid, row in validation.iterrows():
    if asd_hps_valid[rid]:
        group.append('ASD_HPS')
    elif asd_ns_valid[rid]:
        group.append('ASD_NS')
    elif tdc_hps_valid[rid]:
        group.append('TDC_HPS')
    else:
        group.append('TDC_NS')
results_validation = validation.copy()
results_validation['Group'] = group
# Remove missing values
results_validation.replace({col:{-9999:None} for col in results_validation.columns}, inplace=True)

In [ ]:
sbn.barplot(x='Group', y='AGE_AT_SCAN', data=results_validation)

In [ ]:
sbn.barplot(x='Group', y='BV', data=results_validation)

In [ ]:
sbn.barplot(x='Group', y='FD_scrubbed', data=results_validation)

In [ ]:
sbn.barplot(x='Group', y='FIQ', data=results_validation)

In [ ]:
y_pred = scores_s2_arr
HPS_ind = y_pred[:, 1]==1

In [ ]:
# Give me three classes here
asd_hps = [True if HPS_ind[rid]==1 and row['DX_GROUP']=='Autism' else False for rid, row in sample.iterrows()]
asd_ns = [True if HPS_ind[rid]==0 and row['DX_GROUP']=='Autism' else False for rid, row in sample.iterrows()]
tdc = [True if row['DX_GROUP']=='Control' else False for rid, row in sample.iterrows()]
group = list()
for rid, row in sample.iterrows():
    if asd_hps[rid]:
        group.append('ASD_HPS')
    elif asd_ns[rid]:
        group.append('ASD_NS')
    else:
        group.append('TDC')
results = sample.copy()
results['Group'] = group
# Remove missing values
results.replace({col:{-9999:None} for col in results.columns}, inplace=True)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
sbn.barplot(x='Group', y='AGE_AT_SCAN', data=results)

In [ ]:
lm = ols('AGE_AT_SCAN ~ Group',
               data=results).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

In [ ]:
sbn.barplot(x='Group', y='BV', data=results)

In [ ]:
lm = ols('BV ~ Group',
               data=results).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

In [ ]:
sbn.barplot(x='Group', y='FD_scrubbed', data=results)

In [ ]:
lm = ols('FD_scrubbed ~ Group',
               data=results).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

In [ ]:
sbn.barplot(x='Group', y='FIQ', data=results)

In [ ]:
sbn.barplot(x='Group', y='VIQ', data=results)

In [ ]:
sbn.barplot(x='Group', y='PIQ', data=results)

In [ ]:
sbn.barplot(x='Group', y='SRS_RAW_TOTAL', data=results)

In [ ]:
sbn.barplot(x='Group', y='Gotham_Severity', data=results)

In [ ]:
sbn.barplot(x='Group', y='HANDEDNESS_SCORES', data=results)

In [ ]:
f = plt.figure(figsize=(18, 6))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)
for rid, g in enumerate(results.groupby('Group')):
    ax = f.add_subplot(1,3,rid+1)
    g[1]['DSM_IV_TR'].value_counts().plot.pie(ax=ax)
    ax.set_title(g[0])